# 影評資料集(IMDB movie review)情緒分析 

In [ ]:
# 載入相關套件
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# 參數設定
batch_size = 128            # 批量
embedding_output_dims = 15  # 嵌入層輸出維度
max_sequence_length = 300   # 句子最大字數
num_distinct_words = 5000   # 字典
number_of_epochs = 5        # 訓練執行週期
validation_split = 0.20     # 驗證資料比例
verbosity_mode = 1          # 訓練資料訊息顯示程度

In [ ]:
# 載入 IMDB 影評資料集，TensorFlow 已將資料轉為索引值
(x_train, y_train), (x_test, y_test) = imdb.load_data(
    num_words=num_distinct_words)
print(x_train.shape)
print(x_test.shape)

# 長度不足時補 0
padded_inputs = pad_sequences(x_train, maxlen=max_sequence_length
                              , value = 0.0) 
padded_inputs_test = pad_sequences(x_test, maxlen=max_sequence_length
                                   , value = 0.0) 

# 建立模型
model = Sequential()
model.add(Embedding(num_distinct_words, embedding_output_dims, 
                    input_length=max_sequence_length))
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))

# 指定優化器、損失函數
model.compile(optimizer=Adam(), loss=BinaryCrossentropy, metrics=['accuracy'])

# 模型彙總資訊
model.summary()

In [26]:
y_test

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [1]:
# 訓練模型
history = model.fit(padded_inputs, y_train, batch_size=batch_size, 
            epochs=number_of_epochs, verbose=verbosity_mode, 
            validation_split=validation_split)

# 模型評估
test_results = model.evaluate(padded_inputs_test, y_test, verbose=False)
print(f'Loss: {test_results[0]}, Accuracy: {100*test_results[1]}%')

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must sp

(25000,)
(25000,)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 15)           75000     
_________________________________________________________________
lstm (LSTM)                  (None, 10)                1040      
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 76,051
Trainable params: 76,051
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - ETA: 0s - loss: 0.6119 - accuracy: 0.6847

C:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


20000/20000 [==============================] - 143s 7ms/sample - loss: 0.6119 - accuracy: 0.6847 - val_loss: 0.4529 - val_accuracy: 0.8368
Epoch 2/5
20000/20000 [==============================] - 146s 7ms/sample - loss: 0.3766 - accuracy: 0.8580 - val_loss: 0.3708 - val_accuracy: 0.8474
Epoch 3/5
20000/20000 [==============================] - 148s 7ms/sample - loss: 0.2897 - accuracy: 0.8947 - val_loss: 0.3277 - val_accuracy: 0.8694
Epoch 4/5
20000/20000 [==============================] - 148s 7ms/sample - loss: 0.2481 - accuracy: 0.9115 - val_loss: 0.3570 - val_accuracy: 0.8620
Epoch 5/5
20000/20000 [==============================] - 144s 7ms/sample - loss: 0.2160 - accuracy: 0.9264 - val_loss: 0.3258 - val_accuracy: 0.8714
Test results - Loss: 0.33703479669570924 - Accuracy: 86.62800192832947%


In [2]:
# 模型存檔
model.save('LSTM_IMDB.h5')

In [7]:
# 取得字詞與索引的對照表字典
imdb_dict = imdb.get_word_index()
list(imdb_dict.keys())[:10]

['fawn',
 'tsukino',
 'nunnery',
 'sonja',
 'vani',
 'woods',
 'spiders',
 'hanging',
 'woody',
 'trawling']

In [28]:
# 反轉字典，變成索引與字詞的對照表
imdb_dict_reversed = {}
for k, v in imdb_dict.items():
    imdb_dict_reversed[v] = k

In [92]:
# 還原測試資料前兩筆為文字
text = []
for i, line in enumerate(padded_inputs_test[:2]):
    text.append('')
    for j, word in enumerate(line):
        if word != 0:
            text[i] += imdb_dict_reversed[word]+','
        else:
            text[i] += ' ,'
text

[" , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,the,wonder,own,as,by,is,sequence,i,i,and,and,to,of,hollywood,br,of,down,and,getting,boring,of,ever,it,sadly,sadly,sadly,i,i,was,then,does,don't,close,and,after,one,carry,as,by,are,be,and,all,family,turn,in,does,as,three,part,in,another,some,to,be,probably,with,world,and,her,an,have,and,beginning,own,as,is,sequence,",
 " , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,the,as,you,world's,is,quite,br,and,most,that,quest,are,chase,to,being,quickly,of,little,it,time,hell,to,plot,br,of,something,long,put

In [93]:
imdb_dict_reversed[14]

'as'

In [94]:
# 以上述語句測試
X_tokens = []
for line in text:
    tokens = nltk.word_tokenize(line)
    tokens = [token.strip() for token in tokens]
    X_tokens.append(tokens)
    
# 轉為索引值
import numpy as np
X_index = np.zeros((len(text), max_sequence_length))
for i, line in enumerate(X_tokens):
    for j, word in enumerate(line):
        if j >= max_sequence_length:
            break
        if word in imdb_dict:
            X_index[i, j] = imdb_dict[word]


In [95]:
# 長度不足時補 0
padded_inputs = pad_sequences(X_index, maxlen=max_sequence_length, 
                      padding=pad_type, truncating=trunc_type, value = 0.0) 

# 預測
model.predict_classes(padded_inputs)

array([[0],
       [1]])

In [96]:
# 以原資料預測，確認答案相同
model.predict_classes(padded_inputs_test[:2])

array([[0],
       [1]])